In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

In [ ]:
# Step 1: Load and preprocess the CIFAR-10 dataset
# Load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
# Convert class labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Step 2: Load the InceptionV3 model without the top layer
# Use the pre-trained model with ImageNet weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

In [ ]:
# Step 3: Add a custom classification head
# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)
# Add a dense layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)
# Add the final softmax layer with 10 units (for CIFAR-10 classes)
predictions = Dense(10, activation='softmax')(x)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Step 4: Freeze the base model layers to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 5: Compile the model
# Use Adam optimizer with a small learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 6: Train the model
# Resize CIFAR-10 images to 75x75 to match InceptionV3 input requirements
x_train_resized = tf.image.resize(x_train, (75, 75))
x_test_resized = tf.image.resize(x_test, (75, 75))

# Train the model with 20% validation split
history = model.fit(x_train_resized, y_train,
                    validation_split=0.3,
                    batch_size=32,
                    epochs=20)

Epoch 1/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 38s 29ms/step - accuracy: 0.9000 - loss: 0.3794 - val_accuracy: 0.7039 - val_loss: 0.9703
Epoch 2/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - accuracy: 0.9102 - loss: 0.3512 - val_accuracy: 0.6999 - val_loss: 0.9806
Epoch 3/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.9192 - loss: 0.3286 - val_accuracy: 0.6943 - val_loss: 1.0093
Epoch 4/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 25s 22ms/step - accuracy: 0.9287 - loss: 0.3046 - val_accuracy: 0.6912 - val_loss: 1.0290
Epoch 5/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 25s 22ms/step - accuracy: 0.9384 - loss: 0.2778 - val_accuracy: 0.6867 - val_loss: 1.0446
Epoch 6/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.9435 - loss: 0.2622 - val_accuracy: 0.6893 - val_loss: 1.0598
Epoch 7/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.9505 - loss: 0.2459 - val_accuracy: 0.6831 - val_loss: 1.0792
Epoch 8/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9541 -

In [ ]:
# Step 7: Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6188 - loss: 1.2486
Test Accuracy: 0.62
